# Required

In [1]:
import os
import pandas as pd
import zipfile
import numpy as np
from torchvision import transforms
from torchvision.transforms import functional as F
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split

import torch.optim as optim
from torch.optim import SGD
from tqdm import tqdm
import torchvision.transforms as T

import torchvision
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import CocoDetection
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.ops import box_iou

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train_dir = "/content/gdrive/MyDrive/ColabNotebooks/TrainIJCNN2013.zip"
test_dir = "/content/gdrive/MyDrive/ColabNotebooks/TestIJCNN2013.zip"
annotations_file = "/content/gdrive/MyDrive/ColabNotebooks/gt.txt"

In [4]:
local_zip = 'TrainIJCNN2013.zip'
zip_ref = zipfile.ZipFile(train_dir, 'r')
zip_ref.extractall('/content')
zip_ref.close()

local_zip = 'TestIJCNN2013.zip'
zip_ref = zipfile.ZipFile(test_dir, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
train_dir = 'TrainIJCNN2013'
test_dir = 'TestIJCNN2013Download'
shutil.copy(annotations_file, train_dir)
annotations_file = "/content/TrainIJCNN2013/gt.txt"

In [6]:
# Έλεγχος περιεχομένων του αρχείου
with open(annotations_file, 'r') as file:
    for i in range(5):
        print(file.readline())

# Φόρτωση του αρχείου με το σωστό διαχωριστικό
annotations = pd.read_csv(annotations_file, delimiter=',', header=0)

# Εμφάνιση των πρώτων γραμμών και της δομής του DataFrame
print(annotations.head())
print(annotations.columns)
print(annotations.shape)

filename,x1,y1,x2,y2,class

/content/TrainIJCNN2013/00000.ppm,774,411,815,446,11

/content/TrainIJCNN2013/00001.ppm,983,388,1024,432,40

/content/TrainIJCNN2013/00001.ppm,386,494,442,552,38

/content/TrainIJCNN2013/00001.ppm,973,335,1031,390,13

                            filename   x1   y1    x2   y2  class
0  /content/TrainIJCNN2013/00000.ppm  774  411   815  446     11
1  /content/TrainIJCNN2013/00001.ppm  983  388  1024  432     40
2  /content/TrainIJCNN2013/00001.ppm  386  494   442  552     38
3  /content/TrainIJCNN2013/00001.ppm  973  335  1031  390     13
4  /content/TrainIJCNN2013/00002.ppm  892  476  1006  592     39
Index(['filename', 'x1', 'y1', 'x2', 'y2', 'class'], dtype='object')
(852, 6)


In [7]:
# Διάσπαση σε train και test με βάση τα υπάρχοντα directories
train_annotations = annotations  # Χρησιμοποιούμε τα δεδομένα εκπαίδευσης ως έχουν
test_annotations_file = os.path.join(test_dir, annotations_file)
test_annotations = pd.read_csv(annotations_file, delimiter=',', header=0)
test_annotations.columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id']

# Προεπισκόπηση δεδομένων
print(f"Train annotations: {len(train_annotations)}")
print(f"Test annotations: {len(test_annotations)}")

Train annotations: 852
Test annotations: 852


In [9]:
class GTSDBDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None):
        self.annotations = annotations_file
        self.img_dir = img_dir
        self.transform = transform # Προαιρετικοί μετασχηματισμοί (π.χ. κανονικοποίηση)

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        # Φόρτωση εικόνας
        img_path = os.path.join(self.img_dir, self.annotations.iloc[idx, 0])
        image = Image.open(img_path).convert("RGB")

        # Φόρτωση bounding box και ετικέτας
        boxes = self.annotations.iloc[idx, 1:5].values.astype(float)  # Μετατροπή σε float
        boxes = torch.tensor(boxes, dtype=torch.float32).unsqueeze(0)
        label = torch.tensor([self.annotations.iloc[idx, 5]], dtype=torch.int64)

        target = {"boxes": boxes, "labels": label}

        # Εφαρμογή μετασχηματισμών
        if self.transform:
            image = self.transform(image)

        return image, target

In [10]:
# Φόρτωση GTSDB Dataset

transform = T.Compose([
    T.ToTensor(), # Μετατροπή εικόνας σε Tensor
    T.Resize((300, 300)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Κανονικοποίηση
])

train_dataset = GTSDBDataset(train_annotations, train_dir, transform)
test_dataset = GTSDBDataset(test_annotations, test_dir, transform)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

In [22]:
torch.cuda.empty_cache()

In [23]:
import os
import torch
import torchvision.transforms as T
from PIL import Image


# Class name to integer mapping
#CLASS_NAME_TO_ID = {"RBC": 0, "WBC": 1, "Platelets": 2, "Background": 3}


class BCCDDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, label_file, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms
        self.annotations = []

       # Load annotations
        if isinstance(annotations, list):  # If it's a list of dictionaries
            self.annotations = annotations
        elif hasattr(annotations, "iterrows"):  # If it's a DataFrame
            for _, row in annotations.iterrows():
              filename, x1, y1, x2, y2, label = (
                  row["filename"],
                  row["x1"], row["y1"], row["x2"], row["y2"],
                  row["class"]
              )

              # Έλεγχος για μη έγκυρα bounding boxes
              if x1 >= x2 or y1 >= y2:
                  print(f"Παραλείφθηκε μη έγκυρο bounding box: {filename}, {x1}, {y1}, {x2}, {y2}")
                  continue

              self.annotations.append({
                  "filename": filename,
                  "bbox": [float(x1), float(y1), float(x2), float(y2)],
                  "label": int(label)
              })

        else:
            raise ValueError("Annotations should be a DataFrame or a list of dictionaries.")

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        img_path = os.path.join(self.image_dir, annotation["filename"])
        img = Image.open(img_path).convert("RGB")

        # Convert bounding box and label to tensors
        boxes = torch.tensor([annotation["bbox"]], dtype=torch.float32)
        labels = torch.tensor([annotation["label"]], dtype=torch.int64)

        target = {"boxes": boxes, "labels": labels}

        # Apply transforms if provided
        if self.transforms:
            img = self.transforms(img)

        return img, target

# Define transforms
transforms = T.Compose([
    T.ToTensor(),
    T.Resize((300, 300))
])

# Load the dataset
train_dataset = BCCDDataset(train_dir, train_annotations, transforms=transforms)
#val_dataset = BCCDDataset("BCCD/valid", "BCCD/valid/_annotations.csv", transforms=transforms)
test_dataset = BCCDDataset(test_dir, test_annotations, transforms=transforms)

# Example: Access a sample
img, target = train_dataset[0]
print("Image shape:", img.shape)
print("Bounding boxes:", target["boxes"])
print("Labels:", target["labels"])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=4, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

Image shape: torch.Size([3, 300, 300])
Bounding boxes: tensor([[774., 411., 815., 446.]])
Labels: tensor([11])


In [24]:
# Φόρτωση του Faster R-CNN
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# Προσαρμογή του αριθμού κατηγοριών
num_classes = 44  # 43 κατηγορίες + background
in_features = model.roi_heads.box_predictor.cls_score.in_features  # Είσοδος του τελευταίου επιπέδου
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Faster-RCNN

In [25]:
import torch
import torch.optim as optim
from torchvision.ops import box_iou
import time
def calculate_map(pred_boxes, pred_scores, gt_boxes, iou_threshold=0.3):
    """
    Calculate mAP for a batch of predictions and ground truths.
    """
    tp = 0
    fp = 0
    total_gt = len(gt_boxes)

    # Sort predictions by confidence (highest confidence first)
    sorted_indices = torch.argsort(pred_scores, descending=True)
    pred_boxes = pred_boxes[sorted_indices]

    matched_gt = set()

    for i, pred_box in enumerate(pred_boxes):
        best_iou = 0
        best_gt_idx = -1

        # Match with ground truth boxes
        for j, gt_box in enumerate(gt_boxes):
            if j in matched_gt:
                continue
            iou = box_iou(pred_box.unsqueeze(0), gt_box.unsqueeze(0)).item()
            if iou > best_iou:
                best_iou = iou
                best_gt_idx = j

        if best_iou > iou_threshold:
            tp += 1
            matched_gt.add(best_gt_idx)
        else:
            fp += 1

    precision = tp / (tp + fp + 1e-6)
    recall = tp / total_gt
    return precision, recall

def train_model_with_map(model, train_loader, val_loader, device, num_epochs=10, lr=1e-4):
    """
    Train a Faster R-CNN model with mAP calculation and verbose outputs.

    Args:
    - model: PyTorch Faster R-CNN model.
    - train_loader: DataLoader for training set.
    - val_loader: DataLoader for validation set.
    - device: 'cuda' or 'cpu'.
    - num_epochs: Number of epochs.
    - lr: Learning rate.
    """
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.to(device)

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")
        print("-" * 50)

        # Training phase
        model.train()
        epoch_loss = 0
        for batch_idx, (images, targets) in enumerate(train_loader):
          # if batch_idx>3:
            # continue
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            # Forward and compute loss
            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())
            epoch_loss += losses.item()

            # Backward
            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            if (batch_idx + 1) % 10 == 0:
                print(f"Batch [{batch_idx + 1}/{len(train_loader)}] - Loss: {losses.item():.4f}")

        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch + 1}] - Average Loss: {avg_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, targets in val_loader:
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

                # Forward pass to get predictions
                outputs = model(images)

                # Note: Skipping loss calculation in eval mode
                for i, output in enumerate(outputs):
                    print(f"Validation Batch - Image {i+1}: Predicted {len(output['boxes'])} boxes")

        print(f"Epoch [{epoch + 1}] - Validation Done.")

    print("\nTraining Complete!")
    return model

In [18]:
# Initialize the model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 44  # RBC, WBC, Platelets + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the model
trained_model = train_model_with_map(
    model=model,
    train_loader=train_loader,
    val_loader=test_loader,
    device=device,
    num_epochs=1,
    lr=1e-4
)

# Save the trained model
torch.save(trained_model.state_dict(), "faster_rcnn_bccd_with_map.pth")



Epoch 1/1
--------------------------------------------------


OutOfMemoryError: CUDA out of memory. Tried to allocate 158.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 15.06 MiB is free. Process 6379 has 14.73 GiB memory in use. Of the allocated memory 14.56 GiB is allocated by PyTorch, and 49.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Initialize the model architecture
num_classes = 44  # RBC, WBC, Platelets + background
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load trained weights
model.load_state_dict(torch.load("faster_rcnn_bccd_with_map.pth"))
model.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [26]:
# Initialize the model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 44  # RBC, WBC, Platelets + background
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Save the trained model
torch.save(model.state_dict(), "faster_rcnn_bccd_with_map.pth")


In [27]:
# Ορισμός υπερπαραμέτρων
num_epochs = 10
learning_rate = 0.005
momentum = 0.9
weight_decay = 0.0005

# Ορισμός του optimizer
optimizer = SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

# Training Loop
for epoch in range(num_epochs):
    model.train()  # Το μοντέλο σε training mode
    epoch_loss = 0  # Παρακολούθηση της απώλειας
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")  # Γραμμή προόδου

    for images, targets in pbar:
        # Μεταφορά των δεδομένων στη συσκευή (GPU ή CPU)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Μηδενισμός των gradients
        optimizer.zero_grad()

        # Υπολογισμός της απώλειας (loss)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        epoch_loss += losses.item()

        # Οπισθοδιάδοση (backpropagation)
        losses.backward()

        # Ενημέρωση των βαρών
        optimizer.step()

        # Ενημέρωση της γραμμής προόδου
        pbar.set_postfix(loss=losses.item())

    print(f"Epoch {epoch+1} Loss: {epoch_loss:.4f}")

# Αποθήκευση του εκπαιδευμένου μοντέλου
torch.save(model.state_dict(), "faster_rcnn_gtsdb.pth")
print("Το μοντέλο αποθηκεύτηκε ως faster_rcnn_gtsdb.pth")

Epoch 1/10:   0%|          | 0/213 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 15.06 MiB is free. Process 6379 has 14.73 GiB memory in use. Of the allocated memory 14.60 GiB is allocated by PyTorch, and 9.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [28]:
# Initialize the model architecture
num_classes = 44  # RBC, WBC, Platelets + background
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Load trained weights
model.load_state_dict(torch.load("faster_rcnn_bccd_with_map.pth"))
model.eval()  # Set to evaluation mode
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 111MB/s]
<ipython-input-28-c0863af2fe53>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpi

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 7.06 MiB is free. Process 6379 has 14.74 GiB memory in use. Of the allocated memory 14.62 GiB is allocated by PyTorch, and 5.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [49]:
# Λειτουργία αξιολόγησης (inference mode)
model.eval()

# Αποθήκευση των αποτελεσμάτων
all_preds = []
all_targets = []
print(all_targets[:5])
print(all_preds[:5])

with torch.no_grad():
    for images, targets in tqdm(test_loader, desc="Evaluating"):
        # Μεταφορά των δεδομένων στη συσκευή (GPU/CPU)
        images = [image.to(device) for image in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # Προβλέψεις του μοντέλου
        predictions = model(images)

        # Αποθήκευση targets και predictions για αξιολόγηση
        all_preds.extend(predictions)
        all_targets.extend(targets)

[]
[]


Evaluating:   0%|          | 0/27 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB. GPU 0 has a total capacity of 14.75 GiB of which 605.06 MiB is free. Process 2406 has 14.15 GiB memory in use. Of the allocated memory 12.36 GiB is allocated by PyTorch, and 1.65 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [32]:
import cv2
import matplotlib.pyplot as plt
import torchvision.transforms as T

# Map class IDs to class names
ID_TO_CLASS_NAME = {0: "Background", 1: "RBC", 2: "WBC", 3: "Platelets"}

def visualize_predictions(image_path, model, threshold=0.5):
    """
    Visualize predictions on a single image.

    Args:
    - image_path: Path to the input image.
    - model: Trained Faster R-CNN model.
    - threshold: Confidence threshold for displaying predictions.
    """
    # Load and preprocess the image
    img = Image.open(image_path).convert("RGB")
    transform = T.Compose([T.ToTensor()])
    img_tensor = transform(img).to(device)

    # Make predictions
    with torch.no_grad():
        prediction = model([img_tensor])

    # Extract boxes, labels, and scores
    pred_boxes = prediction[0]['boxes'].cpu().numpy()
    pred_scores = prediction[0]['scores'].cpu().numpy()
    pred_labels = prediction[0]['labels'].cpu().numpy()

    # Draw boxes on the original image
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    for box, score, label in zip(pred_boxes, pred_scores, pred_labels):
        if score >= threshold:
            x1, y1, x2, y2 = box.astype(int)
            class_name = ID_TO_CLASS_NAME[label]

            # Draw rectangle and label
            cv2.rectangle(img_cv, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img_cv, f"{class_name}: {score:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # Display the image
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()


In [33]:
import os
import numpy as np
# Test images directory
test_images_dir = 'test_images_dir'
os.makedirs(test_images_dir, exist_ok=True)

# List all test images
test_images = os.listdir(test_images_dir)

# Visualize predictions for a few images
for i in range(5):  # Visualize first 5 images
    image_path = os.path.join(test_images_dir, test_images[i])
    print(f"Visualizing predictions for: {test_images[i]}")
    visualize_predictions(image_path, model, threshold=0.5)


IndexError: list index out of range

In [29]:
# Υπολογισμός mAP για το test set
map_score = calculate_map(all_preds, all_targets)
print(f"Mean Average Precision (mAP): {map_score:.4f}")

TypeError: calculate_map() missing 1 required positional argument: 'gt_boxes'